In [17]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd

# Carregar os dados
df = pd.read_excel('brSerieA.xlsx')

# Calcular o número de vitórias por time
def calcular_vitorias(df, ano=None):
    if ano:
        df = df[df['ano_campeonato'] == ano].copy()  # Fazendo uma cópia explícita para evitar o aviso de erro 'SettingWithCopyWarning'
    df['vencedor'] = df.apply(lambda row: row['time_mandante'] if row['gols_mandante'] > row['gols_visitante'] 
                              else row['time_visitante'] if row['gols_visitante'] > row['gols_mandante'] 
                              else 'Empate', axis=1)
    df_vitorias = df[df['vencedor'] != 'Empate'].groupby('vencedor').size().reset_index(name='vitorias')
    return df_vitorias


# Criar o app
app = Dash(__name__)

anos_disponiveis = sorted(df['ano_campeonato'].unique()) 

app.layout = html.Div(children=[
    html.H1(children='Campeonato Brasileiro Série A'),
    
    # Dropdown para selecionar o ano
    html.Label('Selecione o ano:'),
    dcc.Dropdown(
        id='ano-dropdown',
        options=[{'label': str(ano), 'value': ano} for ano in anos_disponiveis],
        value=anos_disponiveis[0],  # Define o primeiro ano disponível como padrão
        clearable=False
    ),
    
    # Dropdown para selecionar a questão
    html.Label('Selecione a questão:'),
    dcc.Dropdown(
        id='quest-dropdown',  # ID corrigido
        options=[
            {'label': 'Qual time mais venceu?', 'value': 'mais_vitorias'},
            {'label': 'Qual time foi o melhor e pior visitante?', 'value': 'melhor_pior_visitante'},
            {'label': 'Qual time foi o melhor e pior mandante?', 'value': 'melhor_pior_mandante'}
        ],
        value='mais_vitorias',  # Valor padrão corrigido
        clearable=False
    ),

    # Gráfico de vitórias
    dcc.Graph(id='grafico-vitorias')
])


@app.callback(
    Output('grafico-vitorias', 'figure'),
    Input('ano-dropdown', 'value')
)
def atualizar_grafico(ano):
    df_vitorias = calcular_vitorias(df, ano)
    fig = px.bar(df_vitorias, x='vencedor', y='vitorias',
                 color='vencedor',
                 title=f'Número de Vitórias por Time em {ano}')
    return fig

if __name__ == '__main__':
    app.run(debug=True)
